In [ ]:
!pip install --upgrade pip
!pip install gensim
!pip install nltk
!pip install tokenizers
!pip install sentencepiece
!pip install python-bidi
!pip install arabic-reshaper
# !pip install PyArabic
!pip install spacy

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torch.optim.lr_scheduler import StepLR
import pandas as pd
# from pyarabic.araby import tokenize, strip_tashkeel
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import numpy as np
from utilities import *
import textProcessing as tp
import nltk
import gensim
from gensim.models import KeyedVectors
from gensim.models import word2vec
import spacy
# nltk.download('punkt')
# train_text = load_text("dataset/train.txt")
# tp.preprocessing_text(train_text,"train_preprocessed.txt")


In [3]:
# Download via terminal commands
# !wget "https://archive.org/download/aravec2.0/tweet_cbow_300.zip"
# !unzip "tweet_cbow_300.zip"

--2023-12-28 19:54:48--  https://archive.org/download/aravec2.0/tweet_cbow_300.zip
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia803107.us.archive.org/0/items/aravec2.0/tweet_cbow_300.zip [following]
--2023-12-28 19:54:48--  https://ia803107.us.archive.org/0/items/aravec2.0/tweet_cbow_300.zip
Resolving ia803107.us.archive.org (ia803107.us.archive.org)... 207.241.232.157
Connecting to ia803107.us.archive.org (ia803107.us.archive.org)|207.241.232.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 746111232 (712M) [application/zip]
Saving to: ‘tweet_cbow_300.zip’

tweet_cbow_300.zip  100%[===================>] 711.55M   943KB/s    in 15m 22s 

2023-12-28 20:10:10 (791 KB/s) - ‘tweet_cbow_300.zip’ saved [746111232/746111232]

Archive:  tweet_cbow_300.zip
  inflating: tweets_cbow_300         
  inflating: tweets_cbow

In [4]:
# # make a directory called "spacyModel"
# %mkdir spacyModel
# model = gensim.models.Word2Vec.load("tweets_cbow_300")
# # export the word2vec fomart to the directory
# model.wv.save_word2vec_format("./spacyModel/aravec.txt")
# # using `gzip` to compress the .txt file
# !gzip ./spacyModel/aravec.txt
!python -m spacy init vectors ar ./spacyModel/aravec.txt.gz spacy.aravec.model
nlp = spacy.load("./spacy.aravec.model/")

2023-12-28 20:13:42.548291: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-28 20:13:42.548343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-28 20:13:42.549584: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-28 20:13:43.637177: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Creating blank nlp object for language 'ar'
331679it [00:33, 9867.64it/s] 
✔ Successfully converted 331679 vectors
✔ Saved nlp object with vectors to output directory. You can now use
the path to it in your config as the 'vectors' setting 

In [5]:
import gensim
from gensim.models import KeyedVectors
from gensim.models import word2vec
import spacy
from torch import mode

class AraVecEmbbedding():
    def __init__(self,model):
        self.model=model

    def map_words_to_vectors(self,sentence):
        return [(token.text,token.vector) for token in self.model(sentence)]


    def AraVec_wordEmbedding(self,sentence):
        return self.map_words_to_vectors(sentence)


In [8]:
from utilities import *
from torch.utils.data import Dataset, DataLoader
import textProcessing as tp
import torch
import numpy as np
from data_preprocessing import DataPreprocessing
import re
class MyDataset(Dataset):
    def __init__(self,model,dataset_path="dataset/train_preprocessed.txt",T=100):
        self.dataPreprocessor = DataPreprocessing()
        self.T = T
        self.data = load_text(dataset_path)
        self.wordEmbedding=AraVecEmbbedding(model)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        sentence = self.data[idx]
        # extract the label
        labels,sentence= self.dataPreprocessor.extract_diacritics_with_previous_letter('s'+sentence)

        # check sentence length
        if len(sentence) > self.T:
            sentence = sentence[:self.T]
            labels = labels[:self.T]
        else:
            for i in range(self.T - len(sentence)):
                sentence += '0'
                labels.append('0')

        assert len(sentence) == self.T
        assert len(labels) == self.T

        # convert the sentence to embedding
        # Word
        arabic_words_with_spaces =  re.findall(r'\s*\b\w+\b\s*|,\s*|\s*\b\w+\b(?!\s*$)',sentence)
        sentence_char = self.dataPreprocessor.convert_sentence_to_vector(sentence)
        sentence_word_embedding =self.wordEmbedding.AraVec_wordEmbedding(sentence)
        list_of_char = []
        for i, (word, word_embedding) in enumerate(sentence_word_embedding):
          length_arabic = len(arabic_words_with_spaces[i])
          for j in range(length_arabic):
            li = sentence_char[j].astype(float)
            concatenated_array = np.concatenate((li, np.array(word_embedding)), axis=0)
            list_of_char.append(concatenated_array)
        assert len(list_of_char) == self.T
        assert len(labels) == self.T
        # convert the labels to one hot encoding
        labels = self.dataPreprocessor.convert_labels_to_indices(labels)

        # reshape the labels
        labels = labels.reshape(-1,1)

        # convert the sentence and labels to tensors
        sentence = torch.tensor(list_of_char, dtype=torch.float32)
        labels = torch.LongTensor(labels)

        return sentence, labels


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from LSTM import LSTMClassifier
from data_preprocessing import DataPreprocessing

class LSTMTrainer:
    def __init__(self,load=True,epoch = 0,input_size = 39,hidden_size = 128,output_size = 16,batch_size = 512,num_epochs = 20):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
        self.num_epochs = num_epochs

        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.model = LSTMClassifier(self.input_size, self.hidden_size, self.output_size)
        self.current_epoch = 0
        self.current_epoch = epoch
        if load:
            self.load_model(epoch)
        self.model.to(self.device)
        self.dataset = MyDataset(model = nlp,T = 280)
        self.test_dataset = MyDataset(model = nlp,dataset_path="dataset/test_preprocessed.txt",T = 280)
        self.train_dataloader = DataLoader(self.dataset, batch_size=self.batch_size, shuffle=False)
        self.test_dataloader = DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=True)
        self.criterion = nn.CrossEntropyLoss(ignore_index=15)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)
        self.scheduler = StepLR(self.optimizer, step_size=2, gamma=0.5)


    def train(self):
        for epoch in range(0,self.current_epoch):
            self.scheduler.step()
        for epoch in range(self.current_epoch,self.num_epochs):
            self.model.train()
            running_loss = 0.0
            for i, data in enumerate(self.train_dataloader, 0):
                # get the inputs
                inputs, labels = data
                # print("input shape",inputs.size())
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                # zero the parameter gradients
                self.optimizer.zero_grad()
                # forward + backward + optimize
                outputs = self.model(inputs)
                outputs = outputs.view(-1, outputs.shape[-1])
                labels = labels.view(-1)
                loss = self.criterion(outputs, labels)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.5)
                self.optimizer.step()
                # print statistics
                running_loss += loss.item()
                if i % 2 == 0:  # print every 100 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 2))
                    running_loss = 0.0
            self.scheduler.step()
            # save the model if it has the best training loss till now
            self.save_model(epoch+1)
        print('Finished Training')

    def test(self):
        self.model.eval()
        with torch.no_grad():
            running_loss = 0.0
            for i, data in enumerate(self.test_dataloader, 0):
                # get the inputs
                inputs, labels = data
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                # forward + backward + optimize
                outputs = self.model(inputs)
                outputs = outputs.view(-1, outputs.shape[-1])
                labels = labels.view(-1)
                loss = self.criterion(outputs, labels)
                running_loss += loss.item()
            print('Test loss: %.3f' %
                  (running_loss / len(self.test_dataloader)))
    def calcluate_accuracy(self):
        self.model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for i, data in enumerate(self.test_dataloader, 0):
                # get the inputs
                inputs, labels = data
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                # forward + backward + optimize
                outputs = self.model(inputs)
                outputs = outputs.view(-1, outputs.shape[-1])
                labels = labels.view(-1)
                _, predicted = torch.max(outputs.data, 1)
                      # cut the padding
                predicted = predicted[labels != 15]
                labels = labels[labels != 15]
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            print('Accuracy of the network on the test set: %d %%' % (
                    100 * correct / total))
            print('Accuracy of the network on the test set: %f %%' % (
                    100 * correct / total))
        with torch.no_grad():
            correct = 0
            total = 0
            for i, data in enumerate(self.train_dataloader, 0):
                # get the inputs
                inputs, labels = data
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                # forward + backward + optimize
                outputs = self.model(inputs)
                outputs = outputs.view(-1, outputs.shape[-1])
                labels = labels.view(-1)
                _, predicted = torch.max(outputs.data, 1)
                      # cut the padding
                predicted = predicted[labels != 15]
                labels = labels[labels != 15]
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            print('Accuracy of the network on the train set: %d %%' % (
                    100 * correct / total))
            # print floating point accuracy
            print('Accuracy of the network on the train set: %f %%' % (
                    100 * correct / total))
    def save_model(self,epoch):
        torch.save(self.model.state_dict(), "models/lstm_model_"+str(epoch)+".pth")
    def load_model(self,epoch=9):
        self.model.load_state_dict(torch.load("models/lstm_model_"+str(epoch)+".pth"))
        self.model.eval()
        self.current_epoch = epoch



In [18]:
# from LSTM_Training import LSTMTrainer
lstmTrainer = LSTMTrainer(load = False,epoch = 0,input_size = 339,hidden_size = 128,output_size = 16,batch_size = 128,num_epochs = 50)

In [ ]:
lstmTrainer.train()

[4,   253] loss: 0.802
[4,   255] loss: 0.822
[4,   257] loss: 0.805
[4,   259] loss: 0.834
[4,   261] loss: 0.817
[4,   263] loss: 0.802
[4,   265] loss: 0.832
[4,   267] loss: 0.807
[4,   269] loss: 0.810
[4,   271] loss: 0.824
[4,   273] loss: 0.804
[4,   275] loss: 0.821
[4,   277] loss: 0.825
[4,   279] loss: 0.803
[4,   281] loss: 0.813
[4,   283] loss: 0.811
[4,   285] loss: 0.819
[4,   287] loss: 0.821
[4,   289] loss: 0.810
[4,   291] loss: 0.820
[4,   293] loss: 0.806
[4,   295] loss: 0.801
[4,   297] loss: 0.815
[4,   299] loss: 0.823
[4,   301] loss: 0.825
[4,   303] loss: 0.818
[4,   305] loss: 0.804
[4,   307] loss: 0.796
[4,   309] loss: 0.817
[4,   311] loss: 0.822
[4,   313] loss: 0.802
[4,   315] loss: 0.798
[4,   317] loss: 0.802
[4,   319] loss: 0.807
[4,   321] loss: 0.795
[4,   323] loss: 0.800
[4,   325] loss: 0.809
[4,   327] loss: 0.835
[4,   329] loss: 0.808
[4,   331] loss: 0.834
[4,   333] loss: 0.800
[4,   335] loss: 0.826
[4,   337] loss: 0.820
[4,   339] 

In [15]:
lstmTrainer.test()

Test loss: 0.671


In [ ]:
lstmTrainer.calcluate_accuracy()

In [ ]:
lstmTrainer.save_model(2)